# Multi-Objective PID Tuning for Glass Melter

This notebook demonstrates PID controller tuning for glass melting furnace level control using three multi-objective optimization algorithms:

1. **MOPSO** — Multi-Objective Particle Swarm Optimization (Coello et al., IEEE TEVC 2004)
2. **NTA (Weighted Sum)** — Neighbourhood-based Trajectory Algorithm with Dirichlet weights
3. **MONLTA** — Multi-Objective Non-Linear Threshold Accepting (Nahas et al., EPSR 2021)

**Objectives:** Minimize IAE, Overshoot (%), and Settling Time simultaneously.

In [ ]:
import numpy as np
import sys, os
sys.path.insert(0, os.path.join(os.getcwd(), '..'))

from mopso_monlta.config import DEFAULT_PID_BOUNDS, SIM_DEFAULTS
from mopso_monlta.models import simulate_glass_melter, PIDController
from mopso_monlta.optimizers import MOPSO, NTA_WeightedSum, MONLTA
from mopso_monlta.evaluation import (
    evaluate_pid_objectives, get_pareto_front,
    select_best_compromise,
)
from mopso_monlta.visualization import (
    setup_publication_style, COLORS,
    plot_pareto_2d, plot_pareto_3d,
    plot_controller_comparison, plot_convergence,
    plot_monlta_analysis,
)

setup_publication_style()
np.random.seed(42)
print('Imports OK')

## 1. Configuration

PID gains are bounded as $K_p \in [0.1,\,20]$, $K_i \in [0.001,\,5]$, $K_d \in [0,\,10]$.

In [ ]:
bounds = DEFAULT_PID_BOUNDS
n_obj = 3

print(f'PID bounds: {bounds}')
print(f'Simulation defaults: {SIM_DEFAULTS}')

## 2. Open-Loop Glass Melter Response

Visualize the uncontrolled (manual PID) dynamics to understand the plant.

In [ ]:
import matplotlib.pyplot as plt

pid_manual = PIDController(Kp=5.0, Ki=0.5, Kd=1.0,
                           dt=SIM_DEFAULTS['dt'],
                           u_min=0.0, u_max=SIM_DEFAULTS['u_max'])

t, h, u = simulate_glass_melter(
    pid_manual,
    t_end=SIM_DEFAULTS['t_end'],
    dt=SIM_DEFAULTS['dt'],
    setpoint=SIM_DEFAULTS['setpoint'],
    disturbance_time=SIM_DEFAULTS['disturbance_time'],
    disturbance_mag=SIM_DEFAULTS['disturbance_mag'],
)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True)
ax1.plot(t, h, color=COLORS['blue'], linewidth=2)
ax1.axhline(SIM_DEFAULTS['setpoint'], color='gray', ls='--', label='Setpoint')
ax1.set_ylabel('Level $h$ (m)')
ax1.legend(); ax1.set_title('Baseline PID Response')
ax2.plot(t, u, color=COLORS['orange'], linewidth=2)
ax2.set_xlabel('Time (h)'); ax2.set_ylabel('Control $u$ (t/h)')
plt.tight_layout(); plt.show()

## 3. MOPSO Optimization

In [ ]:
mopso = MOPSO(
    objective_func=evaluate_pid_objectives,
    bounds=bounds,
    n_particles=30,
    n_objectives=n_obj,
    archive_size=100,
)
mopso.optimize(n_iterations=60)
print(f'MOPSO archive size: {len(mopso.archive_objectives)}')

## 4. NTA (Weighted Sum) Optimization

In [ ]:
nta = NTA_WeightedSum(
    objective_func=evaluate_pid_objectives,
    bounds=bounds,
    n_objectives=n_obj,
)
nta.optimize(n_weight_vectors=12, n_iterations=200)
print(f'NTA archive size: {len(nta.archive_objectives)}')

## 5. MONLTA Optimization

In [ ]:
monlta = MONLTA(
    objective_func=evaluate_pid_objectives,
    bounds=bounds,
    n_objectives=n_obj,
)
monlta.optimize(n_episodes=5, steps_per_episode=60)
print(f'MONLTA archive size: {len(monlta.archive_objectives)}')

## 6. Pareto Front Comparison

In [ ]:
results_dict = {
    'MOPSO': mopso.archive_objectives,
    'NTA': nta.archive_objectives,
    'MONLTA': monlta.archive_objectives,
}

plot_pareto_2d(results_dict)
plot_pareto_3d(results_dict)

## 7. Best Compromise Selection (TOPSIS)

Select the best compromise PID gains from each method's Pareto front using TOPSIS.

In [ ]:
methods = {
    'MOPSO': (mopso.archive_positions, mopso.archive_objectives),
    'NTA':   (nta.archive_positions,   nta.archive_objectives),
    'MONLTA':(monlta.archive_positions, monlta.archive_objectives),
}

sim_results = []
color_list = [COLORS['blue'], COLORS['orange'], COLORS['green']]
ls_list = ['-', '--', '-.']

for idx, (name, (gains, objs)) in enumerate(methods.items()):
    best_gains, best_obj = select_best_compromise(gains, objs)
    print(f'{name}: Kp={best_gains[0]:.4f}, Ki={best_gains[1]:.4f}, Kd={best_gains[2]:.4f}')
    print(f'       IAE={best_obj[0]:.4f}, OS={best_obj[1]:.2f}%, Ts={best_obj[2]:.4f} h\n')

    pid = PIDController(Kp=best_gains[0], Ki=best_gains[1], Kd=best_gains[2],
                        dt=SIM_DEFAULTS['dt'], u_min=0.0, u_max=SIM_DEFAULTS['u_max'])
    t, h, u = simulate_glass_melter(
        pid, t_end=SIM_DEFAULTS['t_end'], dt=SIM_DEFAULTS['dt'],
        setpoint=SIM_DEFAULTS['setpoint'],
        disturbance_time=SIM_DEFAULTS['disturbance_time'],
        disturbance_mag=SIM_DEFAULTS['disturbance_mag'],
    )
    sim_results.append((name, t, h, u, color_list[idx], ls_list[idx]))

## 8. Closed-Loop Comparison

In [ ]:
plot_controller_comparison(sim_results)

## 9. Convergence Analysis

In [ ]:
plot_convergence(mopso, nta, monlta)

## 10. MONLTA Diagnostics

Visualize the accepting function $H(\zeta)$ and episode-level statistics.

In [ ]:
plot_monlta_analysis(monlta)